In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/new-raw-mooccubex/clean_raw_data.csv
/kaggle/input/new-raw-mooccubex/clean_data_mean.csv
/kaggle/input/new-raw-mooccubex/raw_data.csv
/kaggle/input/new-raw-mooccubex/clean_data_GCN.csv
/kaggle/input/new-raw-mooccubex/FillZero_minmax_baseline/clean_data_minmax_fill-zero.csv
/kaggle/input/new-raw-mooccubex/FillZero_minmax_baseline/clean_week2/val/val_week1_2.csv
/kaggle/input/new-raw-mooccubex/FillZero_minmax_baseline/clean_week2/test/test_week2.csv
/kaggle/input/new-raw-mooccubex/FillZero_minmax_baseline/clean_week2/train/clean_data_week2.csv
/kaggle/input/new-raw-mooccubex/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_2.csv
/kaggle/input/new-raw-mooccubex/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_3.csv
/kaggle/input/new-raw-mooccubex/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_4.csv
/kaggle/input/new-raw-mooccubex/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_1.csv
/kaggle/input/new-raw-mooccubex/FillZero_m

In [2]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras_tuner.tuners import RandomSearch
from sklearn.model_selection import StratifiedKFold
import time
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_auc_score

In [3]:
# Biến global cho base path
BASE_PATH = "/kaggle/input/new-raw-mooccubex/KI_minmax_baseline"
# Tuần và số phần fold
weeks = ['week1', 'week2', 'week3', 'week4']
fold_parts = 5

# Tạo five_fold_files
five_fold_files = {
    week: [
        f"{BASE_PATH}/clean_{week}/train/5-folds/data_part_{i}.csv"
        for i in range(1, fold_parts + 1)
    ]
    for week in weeks
}

# Tạo file_validation
file_validation = {
    'week1': [f"{BASE_PATH}/clean_week1/val/val_week1.csv"],
    'week2': [f"{BASE_PATH}/clean_week2/val/val_week1_2.csv"],
    'week3': [f"{BASE_PATH}/clean_week3/val/val_week1_2_3.csv"],
    'week4': [f"{BASE_PATH}/clean_week4/val/val_week1_2_3_4.csv"]
}

# Tạo file_test
file_test = {
    week: [f"{BASE_PATH}/clean_{week}/test/test_{week}.csv"]
    for week in weeks
}


## Tìm siêu tham số tốt nhất cho từng tuần

In [4]:
# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    return focal_loss_fixed

# Tạo hàm train cho từng tuần
def train_week_model(week_number, file_paths_train, file_validataion):
    # Đọc dữ liệu
    train_data = pd.read_csv(file_paths_train)
    val_data = pd.read_csv(file_validataion)
    
    # Tách đặc trưng và nhãn
    X_train = train_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_train = train_data["classification_encoded"]

    X_val = val_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_val = val_data["classification_encoded"]
    
    # Áp dụng Over-sampling cho dữ liệu huấn luyện bằng SMOTE
    oversampler = SMOTE(sampling_strategy='auto', random_state=42)
    X_train_res, y_train_res = oversampler.fit_resample(X_train, y_train)
    
    # Reshape dữ liệu cho mô hình BiLSTM
    X_train_res = X_train_res.values.reshape(X_train_res.shape[0], X_train_res.shape[1], 1)
    X_val = X_val.values.reshape(X_val.shape[0], X_val.shape[1], 1)
    
    # One-hot encode nhãn
    y_train_res = tf.keras.utils.to_categorical(y_train_res, num_classes=5)
    y_val = tf.keras.utils.to_categorical(y_val, num_classes=5)
    
    def build_model(hp):
        inputs = tf.keras.Input(shape=(X_train_res.shape[1], 1))  # Khởi tạo đầu vào
        
        # Chỉ sử dụng một lớp Bidirectional LSTM
        x = layers.Bidirectional(layers.LSTM(
            units=hp.Int('units_1', min_value=32, max_value=256, step=32),
            return_sequences=False  # Thay đổi thành False vì đây là lớp LSTM duy nhất
        ))(inputs)
        x = layers.Dropout(rate=hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1))(x)
        
        # Lớp đầu ra
        outputs = layers.Dense(5, activation='softmax')(x)
        
        # Khởi tạo mô hình
        model = tf.keras.Model(inputs=inputs, outputs=outputs)
        
        # Compile với Focal Loss
        model.compile(optimizer=tf.keras.optimizers.Adam(
                          learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                      loss=focal_loss(gamma=2., alpha=0.25),
                      metrics=['accuracy'])
        
        return model

    
    # Khởi tạo RandomSearch tuner
    tuner = RandomSearch(
        build_model,
        objective='val_accuracy',
        max_trials=10,
        executions_per_trial=1,
        directory='my_dir',
        project_name=f'bilstm_tuning_week{week_number}'
    )
    
    # Tìm kiếm siêu tham số tốt nhất
    tuner.search(X_train_res, y_train_res,
                 epochs=20,
                 validation_data=(X_val, y_val),
                 batch_size=32)
    
    # Trả về kết quả tối ưu cho tuần
    best_params = tuner.get_best_hyperparameters(num_trials=1)[0]
    return best_params

In [5]:
# Định nghĩa đường dẫn đến dữ liệu cho từng tuần
file_paths_train = {
    week: f"{BASE_PATH}/clean_{week}/train/clean_data_{week}.csv"
    for week in weeks
}

# Định nghĩa file_validation theo quy luật riêng
file_validation = {
    f"week{idx + 1}": f"{BASE_PATH}/clean_week{idx + 1}/val/val_week{'_'.join(str(i) for i in range(1, idx + 2))}.csv"
    for idx in range(len(weeks))
}

In [6]:
# Tìm tham số tốt nhất cho từng tuần
best_params_week1 = train_week_model(1, file_paths_train["week1"], file_validation["week1"])
best_params_week2 = train_week_model(2, file_paths_train["week2"], file_validation["week2"])
best_params_week3 = train_week_model(3, file_paths_train["week3"], file_validation["week3"])
best_params_week4 = train_week_model(4, file_paths_train["week4"], file_validation["week4"])

# In thông tin chi tiết các tham số tối ưu
print("Best Parameters for Week 1:")
for param_name in best_params_week1.values.keys():
    print(f"{param_name}: {best_params_week1.get(param_name)}")

print("\nBest Parameters for Week 2:")
for param_name in best_params_week2.values.keys():
    print(f"{param_name}: {best_params_week2.get(param_name)}")

print("\nBest Parameters for Week 3:")
for param_name in best_params_week3.values.keys():
    print(f"{param_name}: {best_params_week3.get(param_name)}")

print("\nBest Parameters for Week 4:")
for param_name in best_params_week4.values.keys():
    print(f"{param_name}: {best_params_week4.get(param_name)}")


Trial 10 Complete [00h 02m 57s]
val_accuracy: 0.8078096508979797

Best val_accuracy So Far: 0.8175716996192932
Total elapsed time: 00h 28m 16s
Best Parameters for Week 1:
units_1: 192
dropout_1: 0.2
learning_rate: 0.0010331041248433069

Best Parameters for Week 2:
units_1: 160
dropout_1: 0.1
learning_rate: 0.0019315504088487263

Best Parameters for Week 3:
units_1: 96
dropout_1: 0.2
learning_rate: 0.0007078536645991944

Best Parameters for Week 4:
units_1: 256
dropout_1: 0.2
learning_rate: 0.0009780235167626869


## Danh sách tham số tốt nhất của từng tuần

In [7]:
# Danh sách tham số tốt nhất
best_params = {
    "week1": best_params_week1,
    "week2": best_params_week2,
    "week3": best_params_week3,
    "week4": best_params_week4
}

In [8]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import backend as K

# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    
    return focal_loss_fixed

# # Xây dựng mô hình BiLSTM
# def build_Bilstm_model(params, input_shape):
#     inputs = tf.keras.Input(shape=input_shape)  # Định nghĩa đầu vào
    
#     # Bidirectional LSTM layer 1
#     x = layers.Bidirectional(layers.LSTM(
#         units=params.get('units_1'),
#         return_sequences=True
#     ))(inputs)
#     x = layers.Dropout(rate=params.get('dropout_1', 0.2))(x)
    
#     # Bidirectional LSTM layer 2
#     x = layers.Bidirectional(layers.LSTM(
#         units=params.get('units_2', 32),
#         return_sequences=False
#     ))(x)
#     x = layers.Dropout(rate=params.get('dropout_2', 0.2))(x)
    
#     # Lớp đầu ra
#     outputs = layers.Dense(5, activation='softmax')(x)
    
#     # Khởi tạo mô hình
#     model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
#     # Compile với Focal Loss
#     model.compile(optimizer=tf.keras.optimizers.Adam(
#                       learning_rate=params['learning_rate']),
#                   loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
#                   metrics=['accuracy'])
    
#     return model

def build_Bilstm_model(params, input_shape):
    inputs = tf.keras.Input(shape=input_shape)  # Định nghĩa đầu vào
    
    # Chỉ sử dụng một lớp Bidirectional LSTM
    x = layers.Bidirectional(layers.LSTM(
        units=params.get('units_1', 64),
        return_sequences=False  # Đặt thành False vì đây là lớp LSTM cuối cùng
    ))(inputs)
    x = layers.Dropout(rate=params.get('dropout_1', 0.2))(x)
    
    # Lớp đầu ra
    outputs = layers.Dense(5, activation='softmax')(x)
    
    # Khởi tạo mô hình
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    # Compile với Focal Loss
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=params['learning_rate']),
        loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
        metrics=['accuracy']
    )
    
    return model


In [9]:
# Biến lưu kết quả tổng quát
overall_results_5folds = []

# Lặp qua từng tuần
for week, file_paths in five_fold_files.items():
    print(f"\nProcessing {week} with best parameters...")
    params = best_params[week].values
    print(f"best parameters for {week}: {params}")
    
    # Biến lưu kết quả cho từng tuần
    week_results = {
        "week": week,
        "accuracy_per_fold": [],
        "precision_per_label": [],
        "recall_per_label": [],
        "f1_score_per_label": [],
        "auc_roc_per_label": [],    # AUC từng lớp
        "auc_roc_macro": [],        # AUC macro
        "auc_roc_weighted": [],     # AUC weighted (tự tính)
        "precision_macro": [],
        "recall_macro": [],
        "f1_macro": [],
        "precision_weighted": [],
        "recall_weighted": [],
        "f1_weighted": [],
        "confusion_matrices": [],
        "train_times": [],
        "test_times": []
    }

    # Lặp qua từng fold
    for i in range(len(file_paths)):
        print(f"Fold {i+1}: Using file {file_paths[i]} as test set")
        
        # Tải dữ liệu
        test_data = pd.read_csv(file_paths[i])
        train_data = pd.concat([pd.read_csv(file_paths[j]) for j in range(len(file_paths)) if j != i])
        
        # Tách X và y
        X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                           "course_id", "school", "enroll_time", "classification"])
        y_train = to_categorical(train_data['classification_encoded'], num_classes=5)
        
        X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                         "course_id", "school", "enroll_time", "classification"])
        y_test = to_categorical(test_data['classification_encoded'], num_classes=5)

        # Reshape dữ liệu cho LSTM
        X_train = X_train.to_numpy().reshape((X_train.shape[0], 1, X_train.shape[1]))
        X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))
        
        # Xây dựng mô hình với tham số tốt nhất
        input_shape = (X_train.shape[1], X_train.shape[2])
        model = build_Bilstm_model(params, input_shape)
        
        # Bắt đầu tính thời gian huấn luyện
        start_train = time.time()
        model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=32)
        end_train = time.time()
        
        # Bắt đầu tính thời gian kiểm thử
        start_test = time.time()
        y_pred = model.predict(X_test)
        end_test = time.time()
        
        # Tính thời gian và lưu lại
        train_time = end_train - start_train
        test_time = end_test - start_test
        week_results["train_times"].append(train_time)
        week_results["test_times"].append(test_time)

        # Đánh giá mô hình trên tập kiểm thử của fold hiện tại
        _, accuracy = model.evaluate(X_test, y_test, verbose=0)
        week_results["accuracy_per_fold"].append(accuracy)
        
        # Dự đoán
        y_pred_classes = y_pred.argmax(axis=1)
        y_test_classes = y_test.argmax(axis=1)
        
        # Tính các chỉ số cho mỗi fold
        precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='weighted')
        conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
        
        # Tính AUC-ROC
        try:
            # Tính AUC macro và theo từng lớp với OvR
            auc_macro = roc_auc_score(y_test, y_pred, multi_class="ovr", average="macro")
            auc_per_class = roc_auc_score(y_test, y_pred, multi_class="ovr", average=None)
            # Tính AUC weighted: tính trọng số theo số mẫu của từng lớp
            supports = np.bincount(y_test_classes, minlength=5)
            auc_weighted = np.sum(auc_per_class * supports) / np.sum(supports)
        except Exception as e:
            print(f"Lỗi khi tính AUC: {e}")
            auc_macro = np.nan
            auc_per_class = [np.nan] * 5
            auc_weighted = np.nan
            
        # Lưu kết quả của fold hiện tại
        week_results["precision_per_label"].append(precision)
        week_results["recall_per_label"].append(recall)
        week_results["f1_score_per_label"].append(f1)
        week_results["auc_roc_per_label"].append(auc_per_class)  # AUC từng lớp
        week_results["auc_roc_macro"].append(auc_macro)          # AUC macro
        week_results["auc_roc_weighted"].append(auc_weighted)      # AUC weighted
        week_results["confusion_matrices"].append(conf_matrix)
        week_results["precision_macro"].append(precision_macro)
        week_results["recall_macro"].append(recall_macro)
        week_results["f1_macro"].append(f1_macro)
        week_results["precision_weighted"].append(precision_weighted)
        week_results["recall_weighted"].append(recall_weighted)
        week_results["f1_weighted"].append(f1_weighted)

    # Tính trung bình cho từng nhãn
    average_precision_per_label = np.mean(week_results["precision_per_label"], axis=0)
    average_recall_per_label = np.nanmean(week_results["recall_per_label"], axis=0)
    average_f1_per_label = np.nanmean(week_results["f1_score_per_label"], axis=0)
    average_auc_per_label = np.nanmean(week_results["auc_roc_per_label"], axis=0)
    average_confusion_matrix = np.nanmean(week_results["confusion_matrices"], axis=0)
    average_train_time = sum(week_results["train_times"]) / len(week_results["train_times"])
    average_test_time = sum(week_results["test_times"]) / len(week_results["test_times"])
    average_accuracy = np.nanmean(week_results["accuracy_per_fold"])
    average_precision_macro = np.nanmean(week_results["precision_macro"])
    average_recall_macro = np.nanmean(week_results["recall_macro"])
    average_f1_macro = np.nanmean(week_results["f1_macro"])
    average_auc_macro = np.nanmean(week_results["auc_roc_macro"])
    average_precision_weighted = np.nanmean(week_results["precision_weighted"])
    average_recall_weighted = np.nanmean(week_results["recall_weighted"])
    average_f1_weighted = np.nanmean(week_results["f1_weighted"])
    average_auc_weighted = np.nanmean(week_results["auc_roc_weighted"])


    # Tạo DataFrame cho precision, recall, f1-score
    labels = np.unique(y_test_classes)  # Lấy nhãn từ y_test_classes
    metrics_df = pd.DataFrame({
        "Label": labels,
        "Average Precision": average_precision_per_label,
        "Average Recall": average_recall_per_label,
        "Average F1-Score": average_f1_per_label,
        "Average AUC": average_auc_per_label
    })
    
    # Tạo DataFrame cho confusion matrix
    confusion_df = pd.DataFrame(average_confusion_matrix, index=labels, columns=labels)
    # In kết quả Accuracy và Macro metrics
    print("\n=== Average Accuracy ===")
    print(f"{average_accuracy:.4f}")
    print("\n=== Average Macro Metrics ===")
    print(f"Macro Precision: {average_precision_macro:.4f}")
    print(f"Macro Recall: {average_recall_macro:.4f}")
    print(f"Macro F1-Score: {average_f1_macro:.4f}")
    print(f"Macro AUC-ROC: {average_auc_macro:.4f}")
    print("\n=== Average Weighted Metrics ===")
    print(f"Weighted Precision: {average_precision_weighted:.4f}")
    print(f"Weighted Recall: {average_recall_weighted:.4f}")
    print(f"Weighted F1-Score: {average_f1_weighted:.4f}")
    print(f"Weighted AUC-ROC: {average_auc_weighted:.4f}")
    print("\n=== Average Metrics per Label ===")
    print(metrics_df)
    print("\n=== Average Confusion Matrix ===")
    print(confusion_df)
    
    # Cập nhật kết quả cho tuần hiện tại
    week_results.update({
        "average_accuracy": average_accuracy,
        "average_precision_macro": average_precision_macro,
        "average_recall_macro": average_recall_macro,
        "average_f1_macro": average_f1_macro,
        "average_auc_macro": average_auc_macro,
        "average_precision_weighted": average_precision_weighted,
        "average_recall_weighted": average_recall_weighted,
        "average_f1_weighted": average_f1_weighted,
        "average_auc_weighted": average_auc_weighted,
        "average_metrics_df": metrics_df,
        "average_confusion_matrix": confusion_df,
        "average_train_times": average_train_time,
        "average_test_times": average_test_time,
    })
    overall_results_5folds.append(week_results)


Processing week1 with best parameters...
best parameters for week1: {'units_1': 192, 'dropout_1': 0.2, 'learning_rate': 0.0010331041248433069}
Fold 1: Using file /kaggle/input/new-raw-mooccubex/KI_minmax_baseline/clean_week1/train/5-folds/data_part_1.csv as test set
Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.6340 - loss: 0.1366 - val_accuracy: 0.6904 - val_loss: 0.1105
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6859 - loss: 0.1128 - val_accuracy: 0.7202 - val_loss: 0.1056
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7009 - loss: 0.1121 - val_accuracy: 0.7205 - val_loss: 0.1048
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6942 - loss: 0.1112 - val_accuracy: 0.7297 - val_loss: 0.1017
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7071 - loss: 0.1116 - val_accuracy: 0.7335 - val_loss: 0.1024
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7142 - loss: 0.1076 - val_

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


328/328 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.6083 - loss: 0.1433 - val_accuracy: 0.6854 - val_loss: 0.1123
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6875 - loss: 0.1161 - val_accuracy: 0.7208 - val_loss: 0.1086
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6855 - loss: 0.1141 - val_accuracy: 0.7189 - val_loss: 0.1044
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7102 - loss: 0.1081 - val_accuracy: 0.7288 - val_loss: 0.1069
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6971 - loss: 0.1097 - val_accuracy: 0.7132 - val_loss: 0.1020
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7038 - loss: 0.1085 - val_accuracy: 0.7262 - val_loss: 0.1019
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7113 - loss: 0.1092 - val_accuracy: 0.7410 - val_loss: 0.1007
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7203 - loss: 0.1058 - val_accuracy: 0.7353 - val_

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



=== Average Accuracy ===
0.7575

=== Average Macro Metrics ===
Macro Precision: 0.6057
Macro Recall: 0.4811
Macro F1-Score: 0.5051
Macro AUC-ROC: 0.8655

=== Average Weighted Metrics ===
Weighted Precision: 0.7407
Weighted Recall: 0.7575
Weighted F1-Score: 0.7361
Weighted AUC-ROC: 0.8820

=== Average Metrics per Label ===
   Label  Average Precision  Average Recall  Average F1-Score  Average AUC
0      0           0.650314        0.659000          0.649849     0.869423
1      1           0.152727        0.018286          0.032656     0.835032
2      2           0.764833        0.422010          0.534802     0.857699
3      3           0.646450        0.400000          0.451302     0.872868
4      4           0.813965        0.906441          0.857100     0.892658

=== Average Confusion Matrix ===
       0    1     2     3       4
0  395.4  1.6   7.4  11.2   184.4
1   47.2  1.6   0.6   5.4    32.8
2   40.6  0.4  69.4   5.6    48.4
3   28.6  0.8   1.8  66.8    69.0
4  103.2  1.8  13.2  

...

Fold 2: Using file /kaggle/input/new-raw-mooccubex/KI_minmax_baseline/clean_week3/train/5-folds/data_part_2.csv as test set
Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.6268 - loss: 0.1346 - val_accuracy: 0.6981 - val_loss: 0.1107
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6791 - loss: 0.1169 - val_accuracy: 0.7011 - val_loss: 0.1091
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7008 - loss: 0.1108 - val_accuracy: 0.7320 - val_loss: 0.1063
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7085 - loss: 0.1081 - val_accuracy: 0.7255 - val_loss: 0.1026
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7139 - loss: 0.1049 - val_accuracy: 0.7427 - val_loss: 0.1005
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7197 - loss: 0.1049 - val_accuracy: 0.7541 - val_loss: 0.0987
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7339 - loss: 0.0980 - val_accuracy: 0.7488

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold 4: Using file /kaggle/input/new-raw-mooccubex/KI_minmax_baseline/clean_week3/train/5-folds/data_part_4.csv as test set
Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.5822 - loss: 0.1497 - val_accuracy: 0.7056 - val_loss: 0.1099
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6836 - loss: 0.1157 - val_accuracy: 0.7117 - val_loss: 0.1062
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6994 - loss: 0.1107 - val_accuracy: 0.7079 - val_loss: 0.1033
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7090 - loss: 0.1070 - val_accuracy: 0.7204 - val_loss: 0.1039
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7139 - loss: 0.1041 - val_accuracy: 0.7269 - val_loss: 0.1007
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7248 - loss: 0.1050 - val_accuracy: 0.7429 - val_loss: 0.0978
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7161 - loss: 0.1042 - val_accuracy: 0.7410

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold 2: Using file /kaggle/input/new-raw-mooccubex/KI_minmax_baseline/clean_week4/train/5-folds/data_part_2.csv as test set
Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.6325 - loss: 0.1357 - val_accuracy: 0.7003 - val_loss: 0.1139
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6925 - loss: 0.1148 - val_accuracy: 0.7061 - val_loss: 0.1090
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7102 - loss: 0.1084 - val_accuracy: 0.7244 - val_loss: 0.1045
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7132 - loss: 0.1054 - val_accuracy: 0.7408 - val_loss: 0.1046
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7223 - loss: 0.1037 - val_accuracy: 0.7347 - val_loss: 0.1013
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7296 - loss: 0.1022 - val_accuracy: 0.7472 - val_loss: 0.0996
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7321 - loss: 0.1008 - val_accuracy: 0.7457

## Kết quả cross validation trên 5-folds

In [10]:
# Duyệt qua các tuần trong overall_results
for week_result in overall_results_5folds:
    week = week_result["week"]
    average_train_time = np.mean(week_result["train_times"])
    average_test_time = np.mean(week_result["test_times"])
    average_metrics_df = week_result["average_metrics_df"]
    average_accuracy = np.mean(week_results["accuracy_per_fold"])
    average_confusion_matrix = week_result["average_confusion_matrix"]
    
    # In kết quả
    print(f"\n=== Results for {week} ===")
    print(f"Average Accurancy: {average_accuracy}")
    print(f"Average Train Time: {average_train_time:.4f} seconds")
    print(f"Average Test Time: {average_test_time:.4f} seconds")
    print(f"Average AUC Macro: {average_auc_macro}")
    print(f"Average AUC Weighted: {average_auc_weighted}")
    print("\nAverage Precision, Recall, F1-Score, AUC-ROC per Label:")
    print(average_metrics_df)
    print("\nAverage Confusion Matrix:")
    print(average_confusion_matrix)



=== Results for week1 ===
Average Accurancy: 0.7753199815750123
Average Train Time: 87.6042 seconds
Average Test Time: 0.4955 seconds
Average AUC Macro: 0.8801682092830756
Average AUC Weighted: 0.8971314207511372

Average Precision, Recall, F1-Score, AUC-ROC per Label:
   Label  Average Precision  Average Recall  Average F1-Score  Average AUC
0      0           0.650314        0.659000          0.649849     0.869423
1      1           0.152727        0.018286          0.032656     0.835032
2      2           0.764833        0.422010          0.534802     0.857699
3      3           0.646450        0.400000          0.451302     0.872868
4      4           0.813965        0.906441          0.857100     0.892658

Average Confusion Matrix:
       0    1     2     3       4
0  395.4  1.6   7.4  11.2   184.4
1   47.2  1.6   0.6   5.4    32.8
2   40.6  0.4  69.4   5.6    48.4
3   28.6  0.8   1.8  66.8    69.0
4  103.2  1.8  13.2  31.8  1453.4

=== Results for week2 ===
Average Accurancy: 0.

## Kiểm tra trên tập test

In [11]:
# Mảng lưu dữ liệu của các tuần
results = []

def process_week(week_num, best_params, results):
    print(f"\n=== Processing Week {week_num} ===")
    params = best_params[f"week{week_num}"].values
    # Đường dẫn tới dữ liệu tuần tương ứng
    train_path = f"{BASE_PATH}/clean_week{week_num}/train/clean_data_week{week_num}.csv"
    test_path = f"{BASE_PATH}/clean_week{week_num}/test/test_week{week_num}.csv"
    
    # Load dữ liệu
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)
    
    # Tách X và y
    X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                       "course_id", "school", "enroll_time", "classification"])
    y_train = train_data['classification_encoded']
    
    X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                     "course_id", "school", "enroll_time", "classification"])
    y_test = test_data['classification_encoded']

    # Áp dụng SMOTE cho tập huấn luyện
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Chuyển đổi nhãn sang dạng one-hot
    y_train_resampled = to_categorical(y_train_resampled, num_classes=5)
    y_test = to_categorical(y_test, num_classes=5)

    # Reshape dữ liệu cho LSTM
    X_train_resampled = X_train_resampled.to_numpy().reshape((X_train_resampled.shape[0], 1, X_train_resampled.shape[1]))
    X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))

    # Xây dựng mô hình với tham số tốt nhất
    input_shape = (X_train_resampled.shape[1], X_train_resampled.shape[2])
    model = build_Bilstm_model(params, input_shape)
    
    # Huấn luyện mô hình
    start_train = time.time()
    model.fit(X_train_resampled, y_train_resampled, epochs=50, validation_split=0.1, batch_size=32)
    end_train = time.time()
    
    # Kiểm thử mô hình
    start_test = time.time()
    y_pred = model.predict(X_test)
    end_test = time.time()
    
    # Tính thời gian huấn luyện và kiểm thử
    train_time = end_train - start_train
    test_time = end_test - start_test
    
    # Đánh giá mô hình
    y_pred_classes = y_pred.argmax(axis=1)
    y_test_classes = y_test.argmax(axis=1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='macro')
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='weighted')
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
    accuracy = accuracy_score(y_test_classes, y_pred_classes)
    
    # Tính AUC-ROC (với one-vs-rest)
    try:
        auc_macro = roc_auc_score(y_test, y_pred, multi_class="ovr", average="macro")
        auc_per_class = roc_auc_score(y_test, y_pred, multi_class="ovr", average=None)
        # Tính AUC weighted tự tính theo trọng số mẫu của từng lớp
        supports = np.bincount(y_test_classes, minlength=5)
        auc_weighted = np.sum(auc_per_class * supports) / np.sum(supports)
    except Exception as e:
        print(f"Lỗi khi tính AUC: {e}")
        auc_macro = np.nan
        auc_per_class = [np.nan] * 5
        auc_weighted = np.nan

    # Lưu kết quả vào mảng
    results.append({
        "week": week_num,
        "train_time": train_time,
        "test_time": test_time,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "accuracy": accuracy,
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "f1_macro": f1_macro,
        "precision_weighted": precision_weighted,
        "recall_weighted": recall_weighted,
        "f1_weighted": f1_weighted,
        "auc_macro": auc_macro,
        "auc_weighted": auc_weighted,
        "auc_per_class": auc_per_class,
        "confusion_matrix": conf_matrix
    })
    
    # In kết quả chi tiết
    print("\n=== Precision, Recall, F1-Score per Label ===")
    print(pd.DataFrame({
        "Label": np.unique(y_test_classes),
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1
    }))

    print("\n=== Macro Averages & Accuracy ===")
    print(f"Macro Precision: {precision_macro:.4f}")
    print(f"Macro Recall: {recall_macro:.4f}")
    print(f"Macro F1-Score: {f1_macro:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    
    print("\n=== Weighted Averages ===")
    print(f"Weighted Precision: {precision_weighted:.4f}")
    print(f"Weighted Recall: {recall_weighted:.4f}")
    print(f"Weighted F1-Score: {f1_weighted:.4f}")
    
    print("\n=== AUC-ROC ===")
    print(f"AUC Macro: {auc_macro:.4f}")
    print(f"AUC Weighted: {auc_weighted:.4f}")
    print(f"AUC per Label: {auc_per_class}")
    
    print("\n=== Confusion Matrix ===")
    print(pd.DataFrame(conf_matrix, index=np.unique(y_test_classes), columns=np.unique(y_test_classes)))
    
    print(f"\nTrain Time: {train_time:.2f} seconds")
    print(f"Test Time: {test_time:.2f} seconds")

In [12]:
process_week(1, best_params, results)


=== Processing Week 1 ===
Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.4861 - loss: 0.1930 - val_accuracy: 0.5188 - val_loss: 0.2547
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5661 - loss: 0.1709 - val_accuracy: 0.3951 - val_loss: 0.3147
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5879 - loss: 0.1647 - val_accuracy: 0.6164 - val_loss: 0.2274
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5901 - loss: 0.1599 - val_accuracy: 0.6081 - val_loss: 0.2363
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5957 - loss: 0.1579 - val_accuracy: 0.4864 - val_loss: 0.2786
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.6025 - loss: 0.1548 - val_accuracy: 0.6558 - val_loss: 0.2110
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6106 - loss: 0.1518 - val_accuracy: 0.6358 - val_loss: 0.2109
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accurac

In [13]:
process_week(2, best_params, results)


=== Processing Week 2 ===
Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.5049 - loss: 0.1896 - val_accuracy: 0.4590 - val_loss: 0.2405
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.5481 - loss: 0.1751 - val_accuracy: 0.6059 - val_loss: 0.2135
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.5721 - loss: 0.1689 - val_accuracy: 0.5897 - val_loss: 0.1979
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.5868 - loss: 0.1641 - val_accuracy: 0.5463 - val_loss: 0.2629
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.5893 - loss: 0.1611 - val_accuracy: 0.6109 - val_loss: 0.2534
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.5901 - loss: 0.1614 - val_accuracy: 0.6870 - val_loss: 0.1830
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.5903 - loss: 0.1593 - val_accuracy: 0.6465 - val_loss: 0.1930
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accurac

In [14]:
process_week(3, best_params, results)


=== Processing Week 3 ===
Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.4770 - loss: 0.1962 - val_accuracy: 0.4774 - val_loss: 0.2411
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.5699 - loss: 0.1695 - val_accuracy: 0.4280 - val_loss: 0.2606
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.5961 - loss: 0.1594 - val_accuracy: 0.5630 - val_loss: 0.2360
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.6029 - loss: 0.1545 - val_accuracy: 0.6408 - val_loss: 0.2196
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.6113 - loss: 0.1501 - val_accuracy: 0.5173 - val_loss: 0.2455
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.6149 - loss: 0.1485 - val_accuracy: 0.7079 - val_loss: 0.1806
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.6165 - loss: 0.1473 - val_accuracy: 0.6206 - val_loss: 0.2272
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accurac

In [15]:
process_week(4, best_params, results)


=== Processing Week 4 ===
Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.5059 - loss: 0.1866 - val_accuracy: 0.4345 - val_loss: 0.2437
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.5843 - loss: 0.1639 - val_accuracy: 0.4325 - val_loss: 0.2949
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.5929 - loss: 0.1590 - val_accuracy: 0.6114 - val_loss: 0.2139
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.6081 - loss: 0.1526 - val_accuracy: 0.6558 - val_loss: 0.2334
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.6079 - loss: 0.1503 - val_accuracy: 0.6665 - val_loss: 0.1939
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.6163 - loss: 0.1470 - val_accuracy: 0.6939 - val_loss: 0.1919
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.6215 - loss: 0.1439 - val_accuracy: 0.5677 - val_loss: 0.2199
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accurac

In [16]:
# Hiển thị dữ liệu của các tuần
print("\n=== Summary Results for All Weeks ===")
for result in results:
    print(f"Week {result['week']}:")
    print(f"  Train Time: {result['train_time']:.2f} seconds")
    print(f"  Test Time: {result['test_time']:.2f} seconds")
    print(f"  Accurancy: {result['accuracy']}")
    print(f"  Precision: {result['precision']}")
    print(f"  Recall: {result['recall']}")
    print(f"  F1-Score: {result['f1_score']}")
    print(f"  Macro Precision: {result['precision_macro']}")
    print(f"  Macro Recall: {result['recall_macro']}")
    print(f"  Macro F1-Score: {result['f1_macro']}")
    print(f"  Confusion Matrix:\n{result['confusion_matrix']}")
    print("\n=== AUC-ROC ===")
    print(f"AUC Macro: {result['auc_macro']:.4f}")
    print(f"AUC Weighted: {result['auc_weighted']:.4f}")
    print(f"AUC per Label: {result['auc_per_class']}")


=== Summary Results for All Weeks ===
Week 1:
  Train Time: 277.62 seconds
  Test Time: 0.45 seconds
  Accurancy: 0.7012195121951219
  Precision: [0.68387097 0.18376068 0.46280992 0.4962406  0.91805226]
  Recall: [0.56533333 0.7962963  0.54368932 0.62857143 0.77068794]
  F1-Score: [0.6189781  0.29861111 0.5        0.55462185 0.83794038]
  Macro Precision: 0.5489468853787633
  Macro Recall: 0.6609156629561667
  Macro F1-Score: 0.5620302882888364
  Confusion Matrix:
[[212  88  23   8  44]
 [  3  43   1   2   5]
 [ 19  16  56   5   7]
 [  5  14   7  66  13]
 [ 71  73  34  52 773]]

=== AUC-ROC ===
AUC Macro: 0.8878
AUC Weighted: 0.8958
AUC per Label: [0.88159051 0.89916982 0.86012027 0.89311308 0.90481303]
Week 2:
  Train Time: 281.88 seconds
  Test Time: 0.43 seconds
  Accurancy: 0.7128048780487805
  Precision: [0.65994236 0.18226601 0.51818182 0.52631579 0.91617473]
  Recall: [0.61066667 0.68518519 0.55339806 0.66666667 0.77367896]
  F1-Score: [0.63434903 0.28793774 0.53521127 0.588235